In [1]:
import json
import sys
import os

from easydict import EasyDict as edict
import pymysql as sql

## 하위 폴더에 있는 패키지 로딩을 위한 밑작업
SEP       = os.path.sep
MISC_PATH = SEP.join(os.getcwd().split(SEP)[:-2])
ROOT_PATH = SEP.join(os.getcwd().split(SEP)[:-4])
sys.path.append(MISC_PATH)

from misc.config import *
from misc.utils import *

[INFO] config.json 파일을 로딩합니다.
[INFO] ports.json 파일을 로딩합니다.


In [2]:
host   = CONFIGS.global_host
port   = PORTS.sql_port
user   = CONFIGS.sql_user
passwd = CONFIGS.sql_passwd

In [3]:
## DB 서버에 연결
conn   = sql.connect(host     = host, user = user, port = port,
                     password = passwd, db = 'study')

cursor = conn.cursor()

## **1. 제약**
### 1-1. 무결성 관리 
- 데이터 베이스에 데이터가 결함없이 완벽한 상태를 무결성이라한다.
    - 무결성이 깨지면 데이터를 사용하는 응용 프로그램이 제대로 돌아갈 수 없다.
    - 버그에 의해 데이터가 깨질 수도 있는데, 오류에 대처하는 방어 체계를 갖춰야 한다.
      
- 데이터를 처리하는 단계별로 서버측과 클라이언트측이 각자의 무결성 관리 정책을 제공한다.
    - 클라이언트 측에는 입력한 정보에 맞게 라디오 버튼, 리스트 박스 등의 컨트럴로 오입력을 차단한다.
    - 서버 측에서는 DBMS가 규칙을 기억하고 있다가 비정상 데이터의 입력을 거부한다.
    - 적용 범위에 따라 세가지로 구분된다.
      
     |무결성|설명|
     |:--|:--|
     |컬럼 무결성|컬럼 하나에 저장되는 원자적인 값을 점검한다.|
     |엔티티 무결성|레코드끼리 중복값을 가지지 않도록 하여 유일한 식별자를 관리한다.|
     |참조 무결성|테이블 간 관계를 구성하는 키가 항상 유효하도록 관리하며 외래키 제약으로 관리한다.|

- 테이블을 생성할 때 쿼리문에서 Null, Not null, primiary key로 제약을 지정할 수 있다.

In [4]:
## 아래 쿼리문은 데이터의 타입이 달라 에러가 발생한다.
try:
    values  = [['"평양"', '"넓음"', '"많음"', '"n"', '"북한"']]
    insert_(cursor, table_name = 'tCity', values = values)
except Exception as e:
    print(e)

[query] insert into tCity values ("평양", "넓음", "많음", "n", "북한");
[ERROR] (1366, "Incorrect integer value: '넓음' for column `study`.`tCity`.`AREA` at row 1")
[ERROR] 쿼리에 문제가 발생하였습니다.


### 1-2. NULL 허용
- NULL은 아무것도 입력되어 있지 않은 것이며 알 수 없거나, 결정되지 않은 특수한 상태를 의미한다.
    - 반드시 입력해야 하는 필수 필드는 NOT NULL을 지정해 NULL 값을 허용하지 못하게 한다.
    - NOT NULL을 지정하지 않는 경우에는 기본값으로 NULL 값을 허용한다.
        - null 허용 속성의 기본값은 DBMS에 따라 다르며, DB나 연결 수준에서 변경할 수도 있다.
        - 필드 선언문 끝에 항상 Null, Not null을 명시해주는 것이 좋다. 

~~~SQL
    create table tCity(
        name CHAR(10) PRIMARY KEY,
        area INT NULL,
        popu INT NULL,
        metro CHAR(1) NOT NULL,
        region CHAR(6) NOT NULL
    );
~~~
- 위 커리문에서는 name, metro, region 필드는 primary key 혹은 not null  
  속성을 가지고 있어 값을 무조건 입력해야한다.

In [5]:
## area와 popu는 값 생략이 가능하여, 아래 두 쿼리문은 정상 작동한다.
column = 'name, popu, metro, region'
values = [["'울산'", "114", "'y'", "'경상'"]]
insert_(cursor, table_name = 'tCity', column = column, values = values)

column = 'name, metro, region'
values = [["'삼척'", "'n'", "'강원'"]]
insert_(cursor, table_name = 'tCity', column = column, values = values)

select(cursor, '[쿼리 삽입 확인]', table_name = 'tCity')

[query] insert into tCity (name, popu, metro, region) values ('울산', 114, 'y', '경상');
[INFO] 데이터 삽입 완료 

[query] insert into tCity (name, metro, region) values ('삼척', 'n', '강원');
[INFO] 데이터 삽입 완료 

[query] select * from tCity;
[[쿼리 삽입 확인]]
('부산', 765, 342, 'y', '경상')
('삼척', None, None, 'n', '강원')
('서울', 605, 974, 'y', '경기')
('순천', 910, 27, 'n', '전라')
('오산', 42, 21, 'n', '경기')
('울산', None, 114, 'y', '경상')
('전주', 205, 65, 'n', '전라')
('청주', 940, 83, 'n', '충청')
('춘천', 1116, 27, 'n', '강원')
('홍천', 1819, 7, 'n', '강원')




In [6]:
## 아래 쿼리문은 null 값이 허용되지 않은 필드에도 null 값을 삽입하려고 해 에러가 발생한다.
column = 'area, popu, metro, region'
values = [["456", "123", "'n'", "'충청'"]]
insert_(cursor, table_name = 'tCity', column = column, values = values)

[query] insert into tCity (area, popu, metro, region) values (456, 123, 'n', '충청');
[ERROR] (1364, "Field 'NAME' doesn't have a default value")
[ERROR] 쿼리에 문제가 발생하였습니다.


In [9]:
cursor.execute('''
    create table if not exists tNullable(
        name char(10) not null,
        age int
    )
''')

0

In [13]:
## 아래 쿼리문들을 통해 null 값을 지정하지 않아도
## 기본적으로 null을 허용함을 알 수 있다.
values = [["'흥부'", "36"]]
insert_(cursor, table_name = 'tNullable', values = values)

column = 'name'
values = [["'놀부'"]]
insert_(cursor, table_name = 'tNullable', values = values, column = column)

select(cursor, '[null 기본값 확인]', table_name = 'tNullable')

[query] insert into tNullable values ('흥부', 36);
[INFO] 데이터 삽입 완료 

[query] insert into tNullable (name) values ('놀부');
[INFO] 데이터 삽입 완료 

[query] select * from tNullable;
[[null 기본값 확인]]
('흥부', 36)
('놀부', None)




### 1-3. 기본값
- null 허용속성은 필드값이 존재하는지 점검해야하고, 보통 값과 다루는 방식이 달라 DB속도가 저해된다.
    - 필드에 null을 허용 속성을 지정하기 보다 기본값을 사용하는 것이 좋다.
    - 보통 수치형은 0으로, 문자열은 ' '이나 'N/A'로 기본값을 많이 사용한다.  

In [14]:
conn.commit()
conn.close()

# **99. 참고자료**
## **99-1. 도서** 
- 소문난 명강의 - 김상형의 SQL 정복 | 김상형 저 / 한빛 미디어

## **99-2. 논문, 학술지**

## **99-3. 웹 사이트**

## **99-4. 데이터셋 출처**
